In [523]:
import duckdb
import pandas as pd
from fredapi import Fred
import certifi
import os
from datetime import datetime, timedelta, date
from config import FRED_API_KEY
os.environ['SSL_CERT_FILE'] = certifi.where() #API Key security requirements
fred = Fred(api_key=FRED_API_KEY)
today_date = date.today().strftime('%Y-%m-%d') #Fetch Date

In [524]:
# Master List of FRED Indexes:
series_ids = [
    "DCOILWTICO", "GASREGW", "DPROPANEMBTX", "CPIAUCSL", "PPIACO", "PCE", "CPILFESL", "UMCSENT", "PCEPILFE", "MICH",
    "PAYEMS", "UNRATE", "CES0500000003", "CIVPART", "JTSJOL", "ICSA", "ADPWNUSNERSA", "U6RATE", "AWHNONAG", "FEDFUNDS", 
    "DFEDTAR", "IOER", "DISCOUNT", "DGS10", "SOFR", "PRIME", "TB3MS", "CPFF", "CSUSHPINSA", "HSN1F", "PERMIT", "HOUST", 
    "USSTHPI", "EXHOSLUSM495S", "MORTGAGE30US", "RRVRUSQ156N", "RHORUSQ156N", "HOSINVUSM495N"
]

# Energy Indexes:
energy_indexes =  ['DCOILWTICO', 'GASREGW', 'DPROPANEMBTX']
# CPI/PPI/Consumer Sentiment Indexes:
cpi_ppi_consumer_sentiment_indexes =  ['CPIAUCSL', 'PPIACO', 'PCE', 'CPILFESL', 'UMCSENT', 'PCEPILFE', 'MICH']
# Employment Indexes:
employment_indexes =  ['PAYEMS', 'UNRATE', 'CES0500000003', 'CIVPART', 'JTSJOL', 'ICSA', 'ADPWNUSNERSA', 'U6RATE', 'AWHNONAG']
# Federal Funds Indexes:
federal_funds_indexes =  ['FEDFUNDS', 'DFEDTAR', 'IOER', 'DISCOUNT', 'DGS10', 'SOFR', 'PRIME', 'TB3MS', 'CPFF']
# Housing Indexes:
housing_indexes =  ['CSUSHPINSA', 'HSN1F', 'PERMIT', 'HOUST', 'USSTHPI', 'EXHOSLUSM495S', 'MORTGAGE30US', 'RRVRUSQ156N', 'RHORUSQ156N', 'HOSINVUSM495N']

index_categories = {
    'energy': energy_indexes,
    'cpi_ppi_sentiment': cpi_ppi_consumer_sentiment_indexes,
    'employment': employment_indexes,
    'federal_funds': federal_funds_indexes,
    'housing': housing_indexes
}

### Fetch Data from the Economic Research Federal Reserve Bank of St. Louis

In [525]:
def fetch_series_data(series_id):
    """
    Fetch series data for a given series ID between start_date and end_date.
    
    Parameters:
    - series_id (str): The FRED series ID.
    
    Returns:
    - pd.Series: The time series data.
    """
    try:
        series_data = fred.get_series(series_id)
        return series_data
    except Exception as e:
        print(f"Error fetching series data for {series_id}: {e}")
        return pd.Series([])

def fetch_metadata(series_id):
    """
    Fetch metadata for a given series ID.
    
    Parameters:
    - series_id (str): The FRED series ID.
    
    Returns:
    - dict: The metadata for the series.
    """
    try:
        metadata = fred.get_series_info(series_id)
        return metadata
    except Exception as e:
        print(f"Error fetching metadata for {series_id}: {e}")
        return {}

def construct_dataframe(series_data, metadata, series_id, fetch_date):
    """
    Construct a DataFrame from series data and metadata.
    
    Parameters:
    - series_data (pd.Series): The time series data.
    - metadata (dict): The metadata for the series.
    - series_id (str): The FRED series ID.
    - fetch_date (str): The date when the data was fetched.
    
    Returns:
    - pd.DataFrame: The constructed DataFrame with data and metadata.
    """
    try:
        df = pd.DataFrame(series_data, columns=['series_index']).reset_index().rename(columns={'index': 'date'})
        df['name'] = metadata.get('title', '')
        df['seriesid'] = series_id
        df['source'] = f'https://fred.stlouisfed.org/series/{series_id}'
        df['units'] = metadata.get('units', '')
        df['seasonal_adjustment'] = metadata.get('seasonal_adjustment', '')
        df['frequency'] = metadata.get('frequency', '')
        df['fred_last_updated_date'] = metadata.get('last_updated', '')
        df['fetch_date'] = fetch_date
        return df
    except Exception as e:
        print(f"Error constructing dataframe for {series_id}: {e}")
        return pd.DataFrame()

def fetch_fred_data(series_ids):
    """
    Fetch data and metadata for a list of FRED series IDs.
    
    Parameters:
    - series_ids (list of str): The FRED series IDs.
    
    Returns:
    - list of pd.DataFrame: A list of DataFrames with data and metadata for each series.
    """
    today_date = date.today().strftime('%Y-%m-%d')
    dataframes = []
    
    for series_id in series_ids:
        series_data = fetch_series_data(series_id)
        if series_data.empty:
            continue
        metadata = fetch_metadata(series_id)
        df = construct_dataframe(series_data, metadata, series_id, today_date)
        dataframes.append(df)
    
    return dataframes


In [526]:
# Fetch data
dataframes = fetch_fred_data(series_ids)

### Connect to DuckDB and Iterate and Place Tables in the DuckDB database

In [527]:
# Creates empty database and shows existing tables
sql_query = ''' 
show tables
''' 

with duckdb.connect('data/fred.db') as con: #using with to automatically close the database
    display(con.sql(sql_query).df())

,name


In [528]:
conn = duckdb.connect('data/fred.db')
for series_id, dataframe in zip(series_ids, dataframes):
    # Convert series ID to lowercase for the table name
    table_name = series_id.lower()
    
    # Register the DataFrame as a virtual table
    conn.register(table_name + "_temp", dataframe)
    
    # Create or replace the physical table from the virtual table
    conn.execute(f'''
    CREATE OR REPLACE TABLE {table_name} AS 
    SELECT * FROM {table_name}_temp
    ''')

    # Optionally, unregister the virtual table to clean up
    conn.unregister(table_name + "_temp")
conn.close()

sql_query = ''' 
show tables
''' 

with duckdb.connect('data/fred.db') as con: #using with to automatically close the database
    display(con.sql(sql_query).df())

,name
0,adpwnusnersa
1,awhnonag
2,ces0500000003
3,civpart
4,cpff
5,cpiaucsl
6,cpilfesl
7,csushpinsa
8,dcoilwtico
9,dfedtar


### Create Bronze Layer of Medallion Architecture

In [529]:
def create_bronze_sql_files(series_ids, target_directory):
    """
    Create .sql files in the target directory for a list of series IDs.
    
    Parameters:
    - series_ids (list of str): The FRED series IDs.
    - target_directory (str): The target directory to store .sql files.
    """
    # Ensure the target directory exists, create if not
    bronze_directory = os.path.join(target_directory, "bronze")
    os.makedirs(bronze_directory, exist_ok=True)
    
    # SQL template
    sql_template = """SELECT 
date AS Date,
series_index AS Series,
name AS Name,
units AS Units,
seasonal_adjustment AS AdjSeas,
frequency AS Freq,
fred_last_updated_date AS LastUpdatedDate,
fetch_date AS FetchDate
FROM {}
"""
    
    # Iterate over series IDs and create SQL files
    for series_id in series_ids:
        file_name = f"bronze_{series_id.lower()}.sql"
        file_path = os.path.join(bronze_directory, file_name)
        
        # Format the SQL query for the current series ID
        sql_query = sql_template.format(series_id.lower())
        
        # Write the SQL query to a file
        with open(file_path, 'w') as sql_file:
            sql_file.write(sql_query)
            
        print(f"SQL file created: {file_path}")

In [530]:
target_directory = "/Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models"
create_bronze_sql_files(series_ids, target_directory)

SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_dcoilwtico.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_gasregw.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_dpropanembtx.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_cpiaucsl.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_ppiaco.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_pce.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_cpilfesl.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_umcsent.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economi

In [531]:
sql_query = ''' 
show tables
''' 

with duckdb.connect('data/fred.db') as con: #using with to automatically close the database
    display(con.sql(sql_query).df())

,name
0,adpwnusnersa
1,awhnonag
2,ces0500000003
3,civpart
4,cpff
5,cpiaucsl
6,cpilfesl
7,csushpinsa
8,dcoilwtico
9,dfedtar


### Create Silver Layer of Medallion Architecture (Union All)

In [532]:
def create_silver_sql_union_all_files(index_categories, target_directory):
    """
    Create .sql files in the 'silver' subdirectory for unioning tables related to each category.

    Parameters:
    - index_categories (dict of list): A dictionary where each key is a category name and each value is a list of series IDs.
    - target_directory (str): The target directory to store .sql files.
    """
    # Ensure the target directory exists, create if not
    silver_directory = os.path.join(target_directory, "silver", "unionall")
    os.makedirs(silver_directory, exist_ok=True)

    # Iterate over each category and create SQL files
    for category_name, series_ids in index_categories.items():
        file_name = f"silver_{category_name}_unionall.sql"
        file_path = os.path.join(silver_directory, file_name)
        
        # Start the SQL query for UNION ALL
        sql_queries = []
        for series_id in series_ids:
            table_name = series_id.lower()
            sql_query = f"SELECT Date, \
            \nSeries, \
            \nName, \
            \nUnits, \
            \nAdjSeas, \
            \nFreq, \
            \nLastUpdatedDate, \
            \nFetchDate \
            \nFROM {{{{ref('bronze_%s')}}}}" % table_name
            sql_queries.append(sql_query)
        union_sql = " \nUNION ALL \n".join(sql_queries)
        
        # Write the SQL query to a file
        with open(file_path, 'w') as sql_file:
            sql_file.write(union_sql)
            
        print(f"SQL file created: {file_path}")

In [533]:
target_directory = "/Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models"
create_silver_sql_union_all_files(index_categories, target_directory)

SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/unionall/silver_energy_unionall.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/unionall/silver_cpi_ppi_sentiment_unionall.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/unionall/silver_employment_unionall.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/unionall/silver_federal_funds_unionall.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/unionall/silver_housing_unionall.sql


In [534]:
sql_query = ''' 
show tables
''' 

with duckdb.connect('data/fred.db') as con: #using with to automatically close the database
    display(con.sql(sql_query).df())

,name
0,adpwnusnersa
1,awhnonag
2,ces0500000003
3,civpart
4,cpff
5,cpiaucsl
6,cpilfesl
7,csushpinsa
8,dcoilwtico
9,dfedtar


### Create Silver Layer of Medallion Architecture (AdjSeas - Boolean)

In [535]:
def create_silver_sql_binaryconversion_files(index_categories, target_directory):
    """
    Create .sql files for transforming the AdjSeas column in the 'silver' subdirectory.

    Parameters:
    - index_categories (dict of list): A dictionary where each key is a category name.
    - target_directory (str): The target directory to store .sql files.
    """
    # Define the target subdirectory for these transformation SQL files
    silver_directory = os.path.join(target_directory, "silver", "binaryconversion")
    os.makedirs(silver_directory, exist_ok=True)

    # SQL template for transforming AdjSeas
    sql_template = """SELECT 
Date,
Series,
Name,
Units,
CASE 
    WHEN AdjSeas = 'Not Seasonally Adjusted' THEN FALSE 
    WHEN AdjSeas = 'Seasonally Adjusted' THEN TRUE 
    ELSE NULL 
END AS AdjSeas,
Freq,
LastUpdatedDate,
FetchDate
FROM {{{{ref('silver_{category_name}_unionall')}}}}"""

    # Iterate over category names to create transformation SQL files
    for category_name in index_categories.keys():
        # Format file name and path
        file_name = f"silver_{category_name}_binaryconversion.sql"
        file_path = os.path.join(silver_directory, file_name)
        
        # Format the SQL query for the current category
        formatted_sql = sql_template.format(category_name=category_name.lower())
        
        # Write the formatted SQL query to a file
        with open(file_path, 'w') as sql_file:
            sql_file.write(formatted_sql)
            
        print(f"SQL file created: {file_path}")

In [536]:
create_silver_sql_binaryconversion_files(index_categories, target_directory)

SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/binaryconversion/silver_energy_binaryconversion.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/binaryconversion/silver_cpi_ppi_sentiment_binaryconversion.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/binaryconversion/silver_employment_binaryconversion.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/binaryconversion/silver_federal_funds_binaryconversion.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/binaryconversion/silver_housing_binaryconversion.sql


In [537]:
sql_query = ''' 
show tables
''' 

with duckdb.connect('data/fred.db') as con: #using with to automatically close the database
    display(con.sql(sql_query).df())

,name
0,adpwnusnersa
1,awhnonag
2,ces0500000003
3,civpart
4,cpff
5,cpiaucsl
6,cpilfesl
7,csushpinsa
8,dcoilwtico
9,dfedtar


### Create Silver Layer of Medallion Architecture (Category Field Added)

In [538]:
def create_silver_sql_add_category_files(index_categories, target_directory):
    """
    Create .sql files for adding a Category column, referencing the binaryconversion files.

    Parameters:
    - index_categories (dict of list): A dictionary where each key is a category name.
    - target_directory (str): The target directory to store .sql files.
    """
    # Define the target subdirectory for these SQL files
    silver_directory = os.path.join(target_directory, "silver", "addcategory")
    os.makedirs(silver_directory, exist_ok=True)

    # SQL template for adding Category column
    sql_template = """SELECT 
Date,
Series,
Name,
Units,
AdjSeas,
Freq,
LastUpdatedDate,
'{}' AS Category,
FetchDate
FROM {{{{ref('silver_{}_binaryconversion')}}}}"""

    # Iterate over category names to create SQL files
    for category_name in index_categories.keys():
        # Format file name and path
        file_name = f"silver_{category_name}_addcategory.sql"
        file_path = os.path.join(silver_directory, file_name)
        
        # Format the SQL query for the current category
        formatted_sql = sql_template.format(category_name, category_name.lower())
        
        # Write the formatted SQL query to a file
        with open(file_path, 'w') as sql_file:
            sql_file.write(formatted_sql)
            
        print(f"SQL file created: {file_path}")

In [539]:
create_silver_sql_add_category_files(index_categories, target_directory)

SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/addcategory/silver_energy_addcategory.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/addcategory/silver_cpi_ppi_sentiment_addcategory.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/addcategory/silver_employment_addcategory.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/addcategory/silver_federal_funds_addcategory.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/addcategory/silver_housing_addcategory.sql


In [540]:
sql_query = ''' 
show tables
''' 

with duckdb.connect('data/fred.db') as con: #using with to automatically close the database
    display(con.sql(sql_query).df())

,name
0,adpwnusnersa
1,awhnonag
2,ces0500000003
3,civpart
4,cpff
5,cpiaucsl
6,cpilfesl
7,csushpinsa
8,dcoilwtico
9,dfedtar


### Create Silver Layer of Medallion Architecture (Explicit Data Types Assigned)

In [541]:
def create_silver_sql_set_datatypes_files(index_categories, target_directory):
    """
    Create .sql files for setting data types, referencing the addcategory files.

    Parameters:
    - index_categories (dict of list): A dictionary where each key is a category name.
    - target_directory (str): The target directory to store .sql files.
    """
    # Define the target subdirectory for these SQL files
    silver_directory = os.path.join(target_directory, "silver", "setdatatypes")
    os.makedirs(silver_directory, exist_ok=True)

    # SQL template for casting data types
    sql_template = """SELECT 
CAST(Date AS DATETIME(7)) AS Date,
CAST(Series AS FLOAT) AS Series,
CAST(Name AS NVARCHAR(60)) AS Name,
CAST(Units AS NVARCHAR(60)) AS Units,
CAST(AdjSeas AS BOOLEAN) AS AdjSeas,
CAST(Freq AS NVARCHAR(60)) AS Freq,
CAST(LastUpdatedDate AS DATETIME(7)) AS LastUpdatedDate,
CAST(FetchDate AS DATETIME(7)) AS FetchDate
FROM {{{{ref('silver_{category_name}_addcategory')}}}}"""

    # Iterate over category names to create SQL files
    for category_name in index_categories.keys():
        # Format file name and path
        file_name = f"silver_{category_name}_setdatatypes.sql"
        file_path = os.path.join(silver_directory, file_name)
        
        # Format the SQL query for the current category
        formatted_sql = sql_template.format(category_name=category_name.lower())
        
        # Write the formatted SQL query to a file
        with open(file_path, 'w') as sql_file:
            sql_file.write(formatted_sql)
            
        print(f"SQL file created: {file_path}")

In [542]:
create_silver_sql_set_datatypes_files(index_categories, target_directory)

SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/setdatatypes/silver_energy_setdatatypes.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/setdatatypes/silver_cpi_ppi_sentiment_setdatatypes.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/setdatatypes/silver_employment_setdatatypes.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/setdatatypes/silver_federal_funds_setdatatypes.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/setdatatypes/silver_housing_setdatatypes.sql


In [543]:
sql_query = ''' 
show tables
''' 

with duckdb.connect('data/fred.db') as con: #using with to automatically close the database
    display(con.sql(sql_query).df())

,name
0,adpwnusnersa
1,awhnonag
2,ces0500000003
3,civpart
4,cpff
5,cpiaucsl
6,cpilfesl
7,csushpinsa
8,dcoilwtico
9,dfedtar


### Create Gold Layer of Medallion Architecture

In [544]:
def create_gold_sql_files(index_categories, target_directory):
    """
    Create .sql files for the gold layer with a date filter, referencing the setdatatypes files from the silver layer.

    Parameters:
    - index_categories (dict of list): A dictionary where each key is a category name.
    - target_directory (str): The target directory to store .sql files.
    """
    # Define the target directory for the gold SQL files
    gold_directory = os.path.join(target_directory, "gold")
    os.makedirs(gold_directory, exist_ok=True)

    # SQL template for the gold layer query with date filtering
    sql_template = """SELECT 
Date,
Series,
Name,
Units,
AdjSeas,
Freq
FROM {{{{ref('silver_{category_name}_setdatatypes')}}}}
WHERE Date <= CURRENT_DATE AND Date > CURRENT_DATE - INTERVAL '10' YEAR"""

    # Iterate over category names to create SQL files
    for category_name in index_categories.keys():
        # Format file name and path
        file_name = f"gold_{category_name}.sql"
        file_path = os.path.join(gold_directory, file_name)
        
        # Format the SQL query for the current category with date filtering
        formatted_sql = sql_template.format(category_name=category_name.lower())
        
        # Write the formatted SQL query to a file
        with open(file_path, 'w') as sql_file:
            sql_file.write(formatted_sql)
            
        print(f"SQL file created: {file_path}")

In [545]:
create_gold_sql_files(index_categories, target_directory)

SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/gold/gold_energy.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/gold/gold_cpi_ppi_sentiment.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/gold/gold_employment.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/gold/gold_federal_funds.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/gold/gold_housing.sql


In [546]:
sql_query = ''' 
show tables
''' 

with duckdb.connect('data/fred.db') as con: #using with to automatically close the database
    display(con.sql(sql_query).df())

,name
0,adpwnusnersa
1,awhnonag
2,ces0500000003
3,civpart
4,cpff
5,cpiaucsl
6,cpilfesl
7,csushpinsa
8,dcoilwtico
9,dfedtar


In [548]:
sql_query = '''
SELECT * 
FROM gold_cpi_ppi_sentiment
'''

with duckdb.connect('data/fred.db') as con: #using with to automatically close the database
    display(con.sql(sql_query).df())

,Date,Series,Name,Units,AdjSeas,Freq
0,2014-04-01,236.468002,Consumer Price Index for All Urban Consumers: ...,Index 1982-1984=100,True,Monthly
1,2014-05-01,236.917999,Consumer Price Index for All Urban Consumers: ...,Index 1982-1984=100,True,Monthly
2,2014-06-01,237.231003,Consumer Price Index for All Urban Consumers: ...,Index 1982-1984=100,True,Monthly
3,2014-07-01,237.498001,Consumer Price Index for All Urban Consumers: ...,Index 1982-1984=100,True,Monthly
4,2014-08-01,237.460007,Consumer Price Index for All Urban Consumers: ...,Index 1982-1984=100,True,Monthly
...,...,...,...,...,...,...
824,2023-09-01,3.200000,University of Michigan: Inflation Expectation,Percent,False,Monthly
825,2023-10-01,4.200000,University of Michigan: Inflation Expectation,Percent,False,Monthly
826,2023-11-01,4.500000,University of Michigan: Inflation Expectation,Percent,False,Monthly
827,2023-12-01,3.100000,University of Michigan: Inflation Expectation,Percent,False,Monthly


In [549]:
sql_query = '''
SELECT * 
FROM gold_employment
'''

with duckdb.connect('data/fred.db') as con: #using with to automatically close the database
    display(con.sql(sql_query).df())

,Date,Series,Name,Units,AdjSeas,Freq
0,2014-04-01,138298.000000,"All Employees, Total Nonfarm",Thousands of Persons,True,Monthly
1,2014-05-01,138500.000000,"All Employees, Total Nonfarm",Thousands of Persons,True,Monthly
2,2014-06-01,138833.000000,"All Employees, Total Nonfarm",Thousands of Persons,True,Monthly
3,2014-07-01,139076.000000,"All Employees, Total Nonfarm",Thousands of Persons,True,Monthly
4,2014-08-01,139257.000000,"All Employees, Total Nonfarm",Thousands of Persons,True,Monthly
...,...,...,...,...,...,...
1863,2023-10-01,33.799999,Average Weekly Hours of Production and Nonsupe...,Hours,True,Monthly
1864,2023-11-01,33.700001,Average Weekly Hours of Production and Nonsupe...,Hours,True,Monthly
1865,2023-12-01,33.799999,Average Weekly Hours of Production and Nonsupe...,Hours,True,Monthly
1866,2024-01-01,33.500000,Average Weekly Hours of Production and Nonsupe...,Hours,True,Monthly


In [550]:
sql_query = '''
SELECT * 
FROM gold_energy
'''

with duckdb.connect('data/fred.db') as con: #using with to automatically close the database
    display(con.sql(sql_query).df())

,Date,Series,Name,Units,AdjSeas,Freq
0,2014-03-17,98.430000,Crude Oil Prices: West Texas Intermediate (WTI...,Dollars per Barrel,False,Daily
1,2014-03-18,100.080002,Crude Oil Prices: West Texas Intermediate (WTI...,Dollars per Barrel,False,Daily
2,2014-03-19,100.709999,Crude Oil Prices: West Texas Intermediate (WTI...,Dollars per Barrel,False,Daily
3,2014-03-20,99.680000,Crude Oil Prices: West Texas Intermediate (WTI...,Dollars per Barrel,False,Daily
4,2014-03-21,99.970001,Crude Oil Prices: West Texas Intermediate (WTI...,Dollars per Barrel,False,Daily
...,...,...,...,...,...,...
5729,2024-03-05,0.810000,"Propane Prices: Mont Belvieu, Texas",Dollars per Gallon,False,Daily
5730,2024-03-06,0.815000,"Propane Prices: Mont Belvieu, Texas",Dollars per Gallon,False,Daily
5731,2024-03-07,0.803000,"Propane Prices: Mont Belvieu, Texas",Dollars per Gallon,False,Daily
5732,2024-03-08,0.780000,"Propane Prices: Mont Belvieu, Texas",Dollars per Gallon,False,Daily


In [551]:
sql_query = '''
SELECT * 
FROM gold_federal_funds
'''

with duckdb.connect('data/fred.db') as con: #using with to automatically close the database
    display(con.sql(sql_query).df())

,Date,Series,Name,Units,AdjSeas,Freq
0,2014-04-01,0.09,Federal Funds Effective Rate,Percent,False,Monthly
1,2014-05-01,0.09,Federal Funds Effective Rate,Percent,False,Monthly
2,2014-06-01,0.10,Federal Funds Effective Rate,Percent,False,Monthly
3,2014-07-01,0.09,Federal Funds Effective Rate,Percent,False,Monthly
4,2014-08-01,0.09,Federal Funds Effective Rate,Percent,False,Monthly
...,...,...,...,...,...,...
9717,2024-03-06,NaN,3-Month Commercial Paper Minus Federal Funds Rate,Percent,False,Daily
9718,2024-03-07,-0.09,3-Month Commercial Paper Minus Federal Funds Rate,Percent,False,Daily
9719,2024-03-08,-0.01,3-Month Commercial Paper Minus Federal Funds Rate,Percent,False,Daily
9720,2024-03-11,NaN,3-Month Commercial Paper Minus Federal Funds Rate,Percent,False,Daily


In [552]:
sql_query = '''
SELECT * 
FROM gold_housing
'''

with duckdb.connect('data/fred.db') as con: #using with to automatically close the database
    display(con.sql(sql_query).df())

,Date,Series,Name,Units,AdjSeas,Freq
0,2014-04-01,1.629680e+02,S&P CoreLogic Case-Shiller U.S. National Home ...,Index Jan 2000=100,False,Monthly
1,2014-05-01,1.646830e+02,S&P CoreLogic Case-Shiller U.S. National Home ...,Index Jan 2000=100,False,Monthly
2,2014-06-01,1.662120e+02,S&P CoreLogic Case-Shiller U.S. National Home ...,Index Jan 2000=100,False,Monthly
3,2014-07-01,1.671360e+02,S&P CoreLogic Case-Shiller U.S. National Home ...,Index Jan 2000=100,False,Monthly
4,2014-08-01,1.674510e+02,S&P CoreLogic Case-Shiller U.S. National Home ...,Index Jan 2000=100,False,Monthly
...,...,...,...,...,...,...
1133,2023-09-01,1.130000e+06,Existing Home Sales: Housing Inventory,Number of Units,False,Monthly
1134,2023-10-01,1.150000e+06,Existing Home Sales: Housing Inventory,Number of Units,False,Monthly
1135,2023-11-01,1.130000e+06,Existing Home Sales: Housing Inventory,Number of Units,False,Monthly
1136,2023-12-01,9.900000e+05,Existing Home Sales: Housing Inventory,Number of Units,False,Monthly
